In [ ]:
! pip install kaggle


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other


In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#loading csv file data in pandas dataframe
column_names=['target','ids','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names ,encoding='ISO-8859-1')

In [ ]:
twitter_data.shape


(1600000, 6)

In [ ]:
#printing first 5 rows
twitter_data.head()


,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#counting number of missing values in dataset
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


In [ ]:

import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
port_stem=PorterStemmer()

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Download required NLTK data
nltk.download("punkt")
nltk.download("stopwords")

# Initialize stemmer
stemmer = PorterStemmer()

# Define negation words
negation_words = set([
    "not", "no", "n't", "dont", "don't", "doesnt", "doesn't", "isnt", "isn't",
    "wasnt", "wasn't", "didnt", "didn't", "wont", "won't", "cant", "can't"
])

# Create custom stopword list by removing negations from NLTK stopwords
default_stopwords = set(stopwords.words('english'))
custom_stopwords = default_stopwords - negation_words

# Handle negations by tagging next word with "not_"
def handle_negation(text):
    words = word_tokenize(text.lower())
    negation = False
    processed_words = []

    for word in words:
        if word in negation_words:
            negation = True
            continue  # Skip the negation word itself

        if negation:
            word = "not_" + word
            negation = False

        processed_words.append(word)

    return " ".join(processed_words)

# Main preprocessing function
def stemming(content):
    if not content:
        return ""

    content = handle_negation(content)

    # Keep underscores to preserve not_*
    content = re.sub(r'[^a-zA-Z_\s]', ' ', content)
    content = content.lower().split()

    processed_content = []

    for word in content:
        if word.startswith("not_"):  # Keep negated words as-is
            processed_content.append(word)
        elif word not in custom_stopwords:
            processed_content.append(stemmer.stem(word))

    return ' '.join(processed_content)

# Example usage
text = "I don't like this product. It's a waste of money!"
print("Preprocessed:", stemming(text))


Preprocessed: not_like product wast money


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [ ]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset ca not_update facebook text might cri re...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4           nationwideclass not_ not_behaving mad ca not_see
                                 ...                        
1599995                       woke not_school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [ ]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset ca not_update facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y ,random_state=2)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
#CONVERTING TEXTUAL DATA TO NUMERICAL
vectorizer=TfidfVectorizer()
vectorizer.fit(X_train)
X_train=vectorizer.transform(X_train)
X_test=vectorizer.transform(X_test)

In [ ]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9556071 stored elements and shape (1280000, 496134)>
  Coords	Values
  (0, 115996)	0.37748330346111286
  (0, 194193)	0.5264130440966419
  (0, 246954)	0.42056428098082826
  (0, 385657)	0.3576077215955148
  (0, 470788)	0.2734183794475176
  (0, 477279)	0.44819689357650566
  (1, 168936)	1.0
  (2, 83282)	0.3054524972913311
  (2, 115996)	0.4517869978405751
  (2, 131787)	0.19800331501877041
  (2, 136887)	0.2835785512933732
  (2, 139837)	0.20700131446091694
  (2, 158706)	0.18475285946192205
  (2, 186889)	0.15806789302846003
  (2, 280418)	0.24088657963304022
  (2, 297610)	0.16389197173033207
  (2, 303399)	0.16391247778652498
  (2, 439561)	0.3210721726439544
  (2, 440491)	0.1825250697950917
  (2, 442423)	0.1483599074226039
  (2, 467638)	0.32181364064188905
  (2, 477649)	0.32751321391897464
  (3, 60876)	0.5197100589286211
  (3, 159850)	0.2698341424383434
  (3, 166931)	0.44541968449466907
  :	:
  (1279996, 347958)	0.21352419296689315
  

In [ ]:
#Logistic Regression Model Training
model=LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [ ]:
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [ ]:
print('Accuracy on training data : ',training_data_accuracy)

Accuracy on training data :  0.820440625


In [ ]:
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [ ]:
print('Accuracy on test data : ',test_data_accuracy)

Accuracy on test data :  0.797428125


In [ ]:
##saving trained model
import pickle
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
#Use this saved model
loaded_model=pickle.load(open('/content/trained_model.sav','rb'))

In [ ]:
X_new=X_test[200]
print(Y_test[200])
prediction=model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print('Negative Tweet')
else:
  print("Positive Tweet")

1
[1]
Positive Tweet


In [ ]:
import pickle

# Save trained model
with open("sentiment_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Save vectorizer
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

print("Model and vectorizer saved successfully!")



Model and vectorizer saved successfully!


In [ ]:
import joblib

# Save model
joblib.dump(model, "sentiment_model.sav")

# Save vectorizer
joblib.dump(vectorizer, "vectorizer.sav")


['vectorizer.sav']

not_like
Negative
